In [91]:
import codecs, json, glob, os, sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import tensorflow as tf
from sklearn.metrics import confusion_matrix, jaccard_score, classification_report, f1_score
import numpy as np
import pandas as pd
from tqdm import tqdm

from generator import label_mapping
from utils.image_processing import read_resize

from utils.plotting import plot_image_label_prediction, plot_uncertainty_heatmaps, \
    plot_uncertainty_statistics, plot_image, plot_label, plot_predictions

VALIDATION_RESULT_DIR = "/home/olle/PycharmProjects/LODE/workspace/feature_segmentation/cv_runs_results"

models = os.listdir(VALIDATION_RESULT_DIR)

configured_labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 15]


## load json files to evaluate

In [92]:
def read_decode_json(file_path):
    obj_text = codecs.open(file_path, 'r', encoding='utf-8').read()
    
    json_result_file = json.loads(obj_text)
    
    image = np.array(json_result_file["image"])
    prediction = np.array(json_result_file["prediction"])
    
    if len(image.shape) > 3:
        image = image[0, :, :, :]
        
    if len(prediction.shape) > 2:
        prediction = prediction[0, :, :]
    
    json_result_file["image"] = image
    json_result_file["label"] = np.array(json_result_file["label"])
    json_result_file["prediction"] = prediction
    return json_result_file

def read_results(model):
    # set paths
    model_dir = os.path.join(VALIDATION_RESULT_DIR, model)
    json_result_paths = glob.glob(model_dir + f"/*.json")

    result_jsons = {}
    
    for json_file_path in json_result_paths:
        record = json_file_path.split("/")[-1].replace(".json", "")
        
        if "model" in record:
            continue
            
        result_jsons[record] = read_decode_json(json_file_path)
    return result_jsons
        
def plot_results(result_jsons, model):
    records = result_jsons.keys()
    
    model_dir = os.path.join(VALIDATION_RESULT_DIR, model)

    for record in records:
        record_log = result_jsons[record]
        data = [record_log["image"], record_log["label"], record_log["prediction"]]

        # all plotting functions
        plot_image_label_prediction(data, model_dir = model_dir, filename=record, mode="test")
        plot_image(record_log["image"], model_dir = model_dir, filename=record, mode="test")
        plot_label(record_log["label"], model_dir = model_dir, filename=record, mode="test")
        plot_predictions([record_log["prediction"]], model_dir = model_dir, filename=record, mode="test")
        
        
def get_results(result_jsons):
    records = result_jsons.keys()

    f1_record_log = {}
    f1_total_log = {}

    all_labels = []
    all_predictions = []

    for record in records:
        record_log = result_jsons[record]

        labels = record_log["label"].flatten()
        predictions = record_log["prediction"].flatten()

        all_labels.extend(labels.tolist())
        all_predictions.extend(predictions.tolist())

        f1_record_log[record] = f1_score(labels, 
                             predictions, 
                             average = None, 
                             labels = configured_labels, 
                             zero_division=1)

    # calculate for each log
    f1_total_log = f1_score(all_labels, 
                            all_predictions, 
                            average = None, 
                            labels = configured_labels, 
                            zero_division=1)

    # convert to data frames for plotting
    total_f1_pd = pd.DataFrame(f1_total_log).T.replace(1.0, np.NaN)
    record_f1_pd = pd.DataFrame(f1_record_log).T.replace(1.0, np.NaN)
    
    return total_f1_pd, record_f1_pd

In [93]:
columns = np.array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])
validation_log = pd.DataFrame(columns=columns)

from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import time

def fill_validation_log(model):
    jsons = read_results(model)
    plot_results(jsons, model)
    total_r, record_r = get_results(jsons)
    return total_r
    # validation_log = validation_log.append(total_r)

In [ ]:
from joblib import Parallel

results = Parallel(n_jobs=2, verbose=1, backend="multiprocessing")(map(delayed(fill_validation_log), 
                                                                       models))

[Parallel(n_jobs=2)]: Using backend MultiprocessingBackend with 2 concurrent workers.


In [ ]:
columns = np.array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])
validation_log = pd.DataFrame(columns=columns)

for i in range(len(results)):
    validation_log = validation_log.append(pd.DataFrame(results[i]))
    
validation_log["model"] = models[0:3]

In [ ]:
validation_log_long = pd.melt(validation_log, id_vars="model")
validation_log_long = validation_log_long.rename(columns={"variable": "feature", "value":"f1-score"})

validation_log_long = validation_log_long[~validation_log_long.feature.isin([0, 2, 12, 13])]
import seaborn as sns
sns.set(rc={'figure.figsize':(15,7)})
sns.set_context("paper", rc={"font.size":20,"axes.titlesize":20,"axes.labelsize":20})   

sns.set_theme(style="whitegrid")
ax = sns.barplot(x="feature", y="f1-score", hue="model", data=validation_log_long)
